In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.inchi import *
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
import urllib
import string
from multiprocessing import Pool


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.inchi import *
from rdkit import RDLogger
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolsToGridImage
import logging
from multiprocessing import Pool
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from utils import *
from custom_layers import *
from model import *
from plots import *
from evals import *

In [4]:
def predict_rt_for_inchi(ds, pretrained_path):
    print(rt)
   
    p=Pool(8)
    batch_size=64
    ds['smiles'] = p.map(inchi2smiles, ds['inchi'])
    ds=ds[ds.smiles!='NA']
    
    
    ds['mol'] = ds['smiles'].apply(molecule_from_smiles)
    ds['graph'] = p.map(graph_from_molecule, ds['mol'])
    
    atom_features_list = []
    bond_features_list = []
    pair_indices_list = []
    for g in ds.graph:
        atom_features_list.append(g[0])
        bond_features_list.append(g[1])
        pair_indices_list.append(g[2])
        
    
        
    
    X = (np.array(atom_features_list), np.array(bond_features_list), np.array(pair_indices_list))
    print(X[0].shape)
    X = (tf.ragged.constant(X[0], dtype=tf.float32),
         tf.ragged.constant(X[1], dtype=tf.float32),
         tf.ragged.constant(X[2], dtype=tf.int64))
    
    
    
    y = np.zeros(ds.shape[0])
    
    dataset = tf.data.Dataset.from_tensor_slices((X,(y)))
    dataset = dataset.batch(batch_size).map(prepare_batch, -1)
    
    mpnn = MPNNModel_frozen(atom_dim=X[0][0][0].shape[0], bond_dim=X[1][0][0].shape[0], batch_size=batch_size)
    mpnn.load_weights(pretrained_path)
    
    
    Y = mpnn.predict(dataset)
    ds['rt_pred'] = Y
    
    return ds[['inchi', 'rt_pred']]

In [ ]:
ds = pd.read_csv('../data/inchi.csv', sep=';')
pretrained_path = '../models/Out_Final/mpnn_our_final.h5'

In [ ]:
preds =  predict_rt_for_inchi(ds, pretrained_path)